In [1]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.plotting import lag_plot
import matplotlib
matplotlib.style.use('ggplot')
get_ipython().run_line_magic('matplotlib', 'inline')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

plt.show(block=True)

sns.set(style="darkgrid", rc={'figure.figsize':(30, 10)})

from sklearn import preprocessing

In [13]:
bd_path = './'
bd_fallas_filename = 'bd_historialDeFallas.csv'
bd_op_filename = 'bd_variablesOperacionales.csv'
bd_mo_filename = 'bd_monitoreoCondiciones.csv'

df_fallas = pd.read_csv(bd_path + bd_fallas_filename, parse_dates = True, index_col = 'Unnamed: 0')
df_op =  pd.read_csv(bd_path + bd_op_filename, parse_dates = True, index_col = 'time')
df_mo =  pd.read_csv(bd_path + bd_mo_filename, parse_dates = True, index_col = 'time')


print("Numero de registros fallas: ", len(df_fallas.index))
print("Numero de columnas fallas: ", len(df_fallas.columns))

print("Numero de registros operacional: ", len(df_op.index))
print("Numero de columnas operacional: ", len(df_op.columns))

print("Numero de registros monitoreo: ", len(df_mo.index))
print("Numero de columnas monitoreo: ", len(df_mo.columns))

print("FALLAS")
print(df_fallas.head())

#df_op["Unnamed: 0"]
df_op.rename(columns = {"Unnamed: 0": "ID"}, inplace = True)
df_mo.rename(columns = {"Unnamed: 0": "ID"}, inplace = True)
#df_mo["Unnamed: 0"]

print("OPERACIONAL")
print(df_op.head())

print("MONITOREO")
print(df_mo.head())

Numero de registros fallas:  22458
Numero de columnas fallas:  10
Numero de registros operacional:  1578240
Numero de columnas operacional:  411
Numero de registros monitoreo:  2092604
Numero de columnas monitoreo:  8
FALLAS
   semana         fecha inicio            fecha fin      total equipos  \
0     1.0  2016-01-01 00:00:00  2016-01-03 13:42:00  66.700000    STMG   
1     1.0  2016-01-03 00:00:00  2016-01-03 14:42:00   0.566667    STMG   
2     1.0  2016-01-03 00:00:00  2016-01-03 16:06:00   0.083333    STMG   
3     1.0  2016-01-03 00:00:00  2016-01-03 16:39:00   0.550000    STMG   
4     2.0  2016-01-04 00:00:00  2016-01-04 07:15:00   0.366667     CH2   

         modo de falla especialidad  \
0                Cinta     Mecánica   
1             Belt Rip           DO   
2     Reductor 2500 KW    Eléctrica   
3                  AFD    Eléctrica   
4  Detector de Metales           DO   

                                          comentario  falla  partida fallida  
0  Mantención pa

In [14]:
# preliminar

ndups_op = len(df_op[df_op.index.duplicated(keep = 'first')].index)
ndups_mo = len(df_mo[df_mo.index.duplicated(keep = 'first')].index)

print("Numero de duplicados OP: ", ndups_op)
print("Numero de duplicados MO: ", ndups_mo)

Numero de duplicados OP:  180
Numero de duplicados MO:  571627


In [15]:
df_mo[df_mo.index.duplicated(keep = False)].head()
#claramente aca los duplicados responden a la naturaleza de la base de datos

ID        jerarquia              equipo          punto  \
time                                                                          
2016-01-01 01:30:00  22  Est. Mot CV-005         Reductor 22  04 RED HO TEM   
2016-01-01 01:30:00  23  Est. Mot CV-005  Polea Motriz 21-22  02 POL AX TEM   
2016-01-01 01:30:00  24  Est. Mot CV-005         Reductor 22  03 RED VE TEM   
2016-01-01 01:30:00  25  Est. Mot CV-005         Reductor 22  08 RED AX TEM   
2016-01-01 01:30:00  26  Est. Mot CV-005            Motor 21        RPM (2)   

                         valor unidad deteccion  entrada  
time                                                      
2016-01-01 01:30:00  40,743111      C       NaN        1  
2016-01-01 01:30:00  28,771273      C       NaN        1  
2016-01-01 01:30:00  36,046627      C       NaN        1  
2016-01-01 01:30:00  41,184380      C       NaN        1  
2016-01-01 01:30:00   0,000000    RPM       NaN        1

In [16]:
df_op[df_op.index.duplicated(keep = False)].head()
# aca no pasa lo mismo, los duplicados son un error de registro

ID estado_motor_cv701  velocidad_cv701_rpm  \
time                                                                  
2016-05-14 23:00:00  194340        FUNCIONANDO             57.89499   
2016-05-14 23:01:00  194341        FUNCIONANDO             58.83249   
2016-05-14 23:02:00  194342        FUNCIONANDO             59.08773   
2016-05-14 23:03:00  194343        FUNCIONANDO             58.89540   
2016-05-14 23:04:00  194344        FUNCIONANDO             59.19171   

                     velocidad_cv701  pesometro_cv701  pesometro_cv01  \
time                                                                    
2016-05-14 23:00:00         486.7209         6608.606       9415.0190   
2016-05-14 23:01:00         487.1935         6395.840      11151.2900   
2016-05-14 23:02:00         491.7235            0.000      14204.3800   
2016-05-14 23:03:00         496.7946            0.000        302.1363   
2016-05-14 23:04:00         495.8186            0.000       9722.2440   

                     velocidadCero_cv01  velocidadCero_A_cv01  \
time                                                            
2016-05-14 23:00:00            96.44083              99.57212   
2016-05-14 23:01:00            95.32611              98.68792   
2016-05-14 23:02:00            96.64007             100.11580   
2016-05-14 23:03:00            98.05807             100.76550   
2016-05-14 23:04:00            96.37553              99.73907   

                     presionHidroset_cr01  potenciaActiva_A_motor_CH1  ...  \
time                                                                   ...   
2016-05-14 23:00:00              246.6980                    231.5152  ...   
2016-05-14 23:01:00              274.4989                    178.2238  ...   
2016-05-14 23:02:00              211.8319                    127.2124  ...   
2016-05-14 23:03:00              248.5618                    171.8898  ...   
2016-05-14 23:04:00              260.5320                    211.6548  ...   

                     tempDevanadoAfd21_1F3_cv07  tempDevanadoAfd21_2F3_cv07  \
time                                                                          
2016-05-14 23:00:00                    65.87146                    63.42449   
2016-05-14 23:01:00                    65.00000                    62.03732   
2016-05-14 23:02:00                    65.00000                    62.00000   
2016-05-14 23:03:00                    65.00000                    62.00000   
2016-05-14 23:04:00                    65.00000                    62.00000   

                     tempAceite21_cv07  tempRodamiento1Afd21_cv07  \
time                                                                
2016-05-14 23:00:00               79.0                       79.0   
2016-05-14 23:01:00               79.0                       79.0   
2016-05-14 23:02:00               79.0                       79.0   
2016-05-14 23:03:00               79.0                       79.0   
2016-05-14 23:04:00               79.0                       79.0   

                     tempRodamiento2Afd21_cv07  presionFreno_cv07  \
time                                                                
2016-05-14 23:00:00                       62.0           2791.633   
2016-05-14 23:01:00                       62.0           2792.397   
2016-05-14 23:02:00                       62.0           2793.758   
2016-05-14 23:03:00                       62.0           2794.187   
2016-05-14 23:04:00                       62.0           2794.986   

                     tempFreno31_cv07  velocidadCorreaSigno_cv07.1  \
time                                                                 
2016-05-14 23:00:00              17.0                        100.0   
2016-05-14 23:01:00              17.0                        100.0   
2016-05-14 23:02:00              17.0                        100.0   
2016-05-14 23:03:00              17.0                        100.0   
2016-05-14 23:04:00              17.0                        100.0   


In [18]:
# al parecer tenemos el problema lindo de los cambios de hora, pero en este caso tenemos que los datos estan ordenados
df_op[df_op.index.duplicated(keep = False)].head(120)

ID estado_motor_cv701  velocidad_cv701_rpm  \
time                                                                  
2016-05-14 23:00:00  194340        FUNCIONANDO             57.89499   
2016-05-14 23:01:00  194341        FUNCIONANDO             58.83249   
2016-05-14 23:02:00  194342        FUNCIONANDO             59.08773   
2016-05-14 23:03:00  194343        FUNCIONANDO             58.89540   
2016-05-14 23:04:00  194344        FUNCIONANDO             59.19171   
...                     ...                ...                  ...   
2016-05-14 23:55:00  194455        FUNCIONANDO             57.85064   
2016-05-14 23:56:00  194456        FUNCIONANDO             57.82562   
2016-05-14 23:57:00  194457        FUNCIONANDO             57.49632   
2016-05-14 23:58:00  194458        FUNCIONANDO             57.74212   
2016-05-14 23:59:00  194459        FUNCIONANDO             58.16806   

                     velocidad_cv701  pesometro_cv701  pesometro_cv01  \
time                                                                    
2016-05-14 23:00:00         486.7209         6608.606       9415.0190   
2016-05-14 23:01:00         487.1935         6395.840      11151.2900   
2016-05-14 23:02:00         491.7235            0.000      14204.3800   
2016-05-14 23:03:00         496.7946            0.000        302.1363   
2016-05-14 23:04:00         495.8186            0.000       9722.2440   
...                              ...              ...             ...   
2016-05-14 23:55:00         485.1740         6590.202      10505.1600   
2016-05-14 23:56:00         482.3611         8513.432       9567.5850   
2016-05-14 23:57:00         483.5485         7101.361       9052.8160   
2016-05-14 23:58:00         486.0000         7135.370       1631.1010   
2016-05-14 23:59:00         487.1154         5162.839      10900.5400   

                     velocidadCero_cv01  velocidadCero_A_cv01  \
time                                                            
2016-05-14 23:00:00            96.44083              99.57212   
2016-05-14 23:01:00            95.32611              98.68792   
2016-05-14 23:02:00            96.64007             100.11580   
2016-05-14 23:03:00            98.05807             100.76550   
2016-05-14 23:04:00            96.37553              99.73907   
...                                 ...                   ...   
2016-05-14 23:55:00            97.46948              99.97190   
2016-05-14 23:56:00            96.02862              99.66505   
2016-05-14 23:57:00            97.36559              99.85008   
2016-05-14 23:58:00            97.05315             100.34240   
2016-05-14 23:59:00            97.62920             100.25750   

                     presionHidroset_cr01  potenciaActiva_A_motor_CH1  ...  \
time                                                                   ...   
2016-05-14 23:00:00              246.6980                   231.51520  ...   
2016-05-14 23:01:00              274.4989                   178.22380  ...   
2016-05-14 23:02:00              211.8319                   127.21240  ...   
2016-05-14 23:03:00              248.5618                   171.88980  ...   
2016-05-14 23:04:00              260.5320                   211.65480  ...   
...                                   ...                         ...  ...   
2016-05-14 23:55:00              219.5058                    84.11505  ...   
2016-05-14 23:56:00              282.1773                   268.58150  ...   
2016-05-14 23:57:00              230.2318                   167.85490  ...   
2016-05-14 23:58:00              225.5825                   158.28680  ...   
2016-05-14 23:59:00              225.7599                   144.62640  ...   

                     tempDevanadoAfd21_1F3_cv07  tempDevanadoAfd21_2F3_cv07  \
time                                                                          
2016-05-14 23:00:00                    65.87146                    63.42449   
2016-05-14 23:01:00                    65.00000      

In [41]:
# arreglo---> se deja el horario de verano

# primera fecha conflictiva: 2016-05-14
end = '2016-05-14 22:59:59'
target = '2016-05-14 23:00:00'
target2 = '2016-05-14 23:59:59'
dfrom = '2016-05-15 00:00:00'
aux1 = df_op[:end].copy()
aux2 = df_op[target:target2].copy()
aux2_1 = aux2.head(60).copy()
aux2_2 = aux2.tail(60).copy()
aux3 = df_op[dfrom:].copy()

aux = pd.concat([aux2_2, aux3])

aux = aux.shift(60, freq='T')

aux = pd.concat([aux1, aux2_1, aux])


In [42]:
# segunda fecha conflicitiva 13 de agosto--> 2016-08-13 
# como se corrio la serie el agujero debe estar en 2016-08-14 01:00:00 --->02:00:00
# chequeamos primero
start = '2016-08-14 01:00:00'
end   = '2016-08-14 01:59:59'
aux[start:end]

Empty DataFrame
Columns: [ID, estado_motor_cv701, velocidad_cv701_rpm, velocidad_cv701, pesometro_cv701, pesometro_cv01, velocidadCero_cv01, velocidadCero_A_cv01, presionHidroset_cr01, potenciaActiva_A_motor_CH1, posicionCono_CH1, corrientePromedio_CH1, T_ambiente_enfriadores_CH1, presionHidroset_cr701, potencia_CH701, posicionCono_CH2, cap_termica_motorCH2, cap_termica_motorCH1, corrientePromedio_CH2, T_ambiente_enfriadores_CH2, potenciaActivaMotor_cv01A, potenciaActivaMotor_cv01B, corrienteMotor_cv01A, corrienteMotor_cv01B, potencia_M1_cv701, potencia_M2_cv701, corrienteMotorA_cv701_faseA, corrienteMotorB_cv701_faseA, corrienteMotorA_cv701_faseB, corrienteMotorB_cv701_faseB, corrienteMotorA_cv701_fasec, corrienteMotorB_cv701_faseC, alimentacionFeeder_cv05, tonelajePromedio_cv05, inventarioStockPilePlanta, alturaPilaDescarga, velocidadFeeder_05, velocidadFeeder_06, velocidadFeeder_07, velocidadFeeder_08, feeder05_onOff, feeder06_onOff, feeder07_onOff, feeder07_onOff.1, TpH_SAG1, TpH_SAG2, TpH_SAG3, stockPile_nivelSensor1, stockPile_nivelSensor2, stockPile_nivel, stockPile_nivelDesc1, stockPile_nivelDesc2, stockPile_nivelDesc3, stockPile_nivelDesc4, stockPile_nivelDesc5, stockPile_nivelDesc6, stockPile_nivelDesc7, stockPile_nivelDesc8, stockPile_nivel7, stockPile_nivel8, stockPile_nivel9, stockPile_nivel10, stockPileMina_CH1, stockPileMina_CH2, feederSpeed_15_onOff, feederSpeed_16_onOff, feederSpeed_17_onOff, feederSpeed_18_onOff, feederSpeed_15_pct, feederSpeed_16_pct, feederSpeed_17_pct, feederSpeed_18_pct, feederSpeed_19_onOff, feederSpeed_20_onOff, feederSpeed_21_onOff, feederSpeed_22_onOff, feederSpeed_19_pct, feederSpeed_20_pct, feederSpeed_21_pct, feederSpeed_22_pct, feederSpeed_701_onOff, feederSpeed_702_onOff, feederSpeed_703_onOff, feederSpeed_704_onOff, feederSpeed_701_pct, feederSpeed_702_pct, feederSpeed_703_pct, feederSpeed_704_pct, f80_cv01_CH1, f80_cv701_CH2, potenciaMotor_CH1, kTon_CH1, kTon_CH2, OSS_CH1, OSS_CH2, estado_CH1, estado_CH2, estadoPicaRoca_CH1, estadoPicaRoca_CH2, supresorPolvo_CH1, ...]
Index: []

[0 rows x 411 columns]

In [43]:
# agujero confirmado, entonces retrasamos el reloj:
aux1 = aux[:start]
aux2 = aux[start:]
aux2 = aux2.shift(-60, freq='T')
aux = pd.concat([aux1, aux2])

In [44]:
# buscamos la tercera fecha conflicitva
aux[aux.index.duplicated(keep = False)]
# para el 2017 la fecha es 2017-05-13 

ID estado_motor_cv701  velocidad_cv701_rpm  \
time                                                                   
2017-05-13 23:00:00   718500        FUNCIONANDO             59.02935   
2017-05-13 23:01:00   718501        FUNCIONANDO             58.94102   
2017-05-13 23:02:00   718502        FUNCIONANDO             59.31905   
2017-05-13 23:03:00   718503        FUNCIONANDO             59.40068   
2017-05-13 23:04:00   718504        FUNCIONANDO             59.05419   
...                      ...                ...                  ...   
2018-05-12 23:55:00  1242775        FUNCIONANDO             59.08547   
2018-05-12 23:56:00  1242776        FUNCIONANDO             58.66097   
2018-05-12 23:57:00  1242777        FUNCIONANDO             59.00270   
2018-05-12 23:58:00  1242778        FUNCIONANDO             59.78499   
2018-05-12 23:59:00  1242779        FUNCIONANDO             58.48879   

                     velocidad_cv701  pesometro_cv701  pesometro_cv01  \
time                                                                    
2017-05-13 23:00:00         992.2126         5924.008       11439.190   
2017-05-13 23:01:00         990.6889         5690.796        6494.683   
2017-05-13 23:02:00         990.5795         4550.213        6091.824   
2017-05-13 23:03:00         988.8000         7244.377        6119.956   
2017-05-13 23:04:00         987.9750         6609.480        7629.486   
...                              ...              ...             ...   
2018-05-12 23:55:00         988.9272         5027.844        5676.893   
2018-05-12 23:56:00         984.8921         8777.815        6453.686   
2018-05-12 23:57:00         985.4105         7917.861        6667.177   
2018-05-12 23:58:00         988.1536         5648.410        8703.970   
2018-05-12 23:59:00         987.8192         5756.723        8215.501   

                     velocidadCero_cv01  velocidadCero_A_cv01  \
time                                                            
2017-05-13 23:00:00            96.80211              99.21001   
2017-05-13 23:01:00            98.12228             100.06700   
2017-05-13 23:02:00            98.18402              99.75377   
2017-05-13 23:03:00            97.91788              99.60905   
2017-05-13 23:04:00            96.35801              99.30665   
...                                 ...                   ...   
2018-05-12 23:55:00            98.56305              95.53423   
2018-05-12 23:56:00            98.66575              95.63828   
2018-05-12 23:57:00            97.95366              95.38786   
2018-05-12 23:58:00            97.91531              94.77895   
2018-05-12 23:59:00            98.16232              95.00091   

                     presionHidroset_cr01  potenciaActiva_A_motor_CH1  ...  \
time                                                                   ...   
2017-05-13 23:00:00              252.6466                    161.3533  ...   
2017-05-13 23:01:00              316.1599                    777.1628  ...   
2017-05-13 23:02:00              326.9467                    526.4443  ...   
2017-05-13 23:03:00              271.5327                    568.7736  ...   
2017-05-13 23:04:00              234.7638                    126.5586  ...   
...                                   ...                         ...  ...   
2018-05-12 23:55:00              295.7635                    285.2273  ...   
2018-05-12 23:56:00              286.9901                    365.3408  ...   
2018-05-12 23:57:00              269.0976                    221.2858  ...   
2018-05-12 23:58:00              269.1360                    175.0408  ...   
2018-05-12 23:59:00              276.1839                    229.7065  ...   

                     tempDevanadoAfd21_1F3_cv07  tempDevanadoAfd21_2F3_cv07  \
time                                                                          
2017-05-13 23:00:00                    40.61579                    38.95736   
2017-05-13 23:01:00                    40

In [45]:
# tercera fecha conflictiva: 2017-05-13
end = '2017-05-13 22:59:59'
target = '2017-05-13 23:00:00'
target2 = '2017-05-13 23:59:59'
dfrom = '2017-05-14 00:00:00'
aux1 = aux[:end].copy()
aux2 = aux[target:target2].copy()
aux2_1 = aux2.head(60).copy()
aux2_2 = aux2.tail(60).copy()
aux3 = aux[dfrom:].copy()

aux_f = pd.concat([aux2_2, aux3])

aux_f = aux_f.shift(60, freq='T')

aux = pd.concat([aux1, aux2_1, aux_f])

In [46]:
# cuarta fecha conflictiva 2017-08-12, todo esta corrido una hora hacia adelante
# confirmando el agujero:
start = '2017-08-13 01:00:00'
end   = '2017-08-13 01:59:59'
aux[start:end]

Empty DataFrame
Columns: [ID, estado_motor_cv701, velocidad_cv701_rpm, velocidad_cv701, pesometro_cv701, pesometro_cv01, velocidadCero_cv01, velocidadCero_A_cv01, presionHidroset_cr01, potenciaActiva_A_motor_CH1, posicionCono_CH1, corrientePromedio_CH1, T_ambiente_enfriadores_CH1, presionHidroset_cr701, potencia_CH701, posicionCono_CH2, cap_termica_motorCH2, cap_termica_motorCH1, corrientePromedio_CH2, T_ambiente_enfriadores_CH2, potenciaActivaMotor_cv01A, potenciaActivaMotor_cv01B, corrienteMotor_cv01A, corrienteMotor_cv01B, potencia_M1_cv701, potencia_M2_cv701, corrienteMotorA_cv701_faseA, corrienteMotorB_cv701_faseA, corrienteMotorA_cv701_faseB, corrienteMotorB_cv701_faseB, corrienteMotorA_cv701_fasec, corrienteMotorB_cv701_faseC, alimentacionFeeder_cv05, tonelajePromedio_cv05, inventarioStockPilePlanta, alturaPilaDescarga, velocidadFeeder_05, velocidadFeeder_06, velocidadFeeder_07, velocidadFeeder_08, feeder05_onOff, feeder06_onOff, feeder07_onOff, feeder07_onOff.1, TpH_SAG1, TpH_SAG2, TpH_SAG3, stockPile_nivelSensor1, stockPile_nivelSensor2, stockPile_nivel, stockPile_nivelDesc1, stockPile_nivelDesc2, stockPile_nivelDesc3, stockPile_nivelDesc4, stockPile_nivelDesc5, stockPile_nivelDesc6, stockPile_nivelDesc7, stockPile_nivelDesc8, stockPile_nivel7, stockPile_nivel8, stockPile_nivel9, stockPile_nivel10, stockPileMina_CH1, stockPileMina_CH2, feederSpeed_15_onOff, feederSpeed_16_onOff, feederSpeed_17_onOff, feederSpeed_18_onOff, feederSpeed_15_pct, feederSpeed_16_pct, feederSpeed_17_pct, feederSpeed_18_pct, feederSpeed_19_onOff, feederSpeed_20_onOff, feederSpeed_21_onOff, feederSpeed_22_onOff, feederSpeed_19_pct, feederSpeed_20_pct, feederSpeed_21_pct, feederSpeed_22_pct, feederSpeed_701_onOff, feederSpeed_702_onOff, feederSpeed_703_onOff, feederSpeed_704_onOff, feederSpeed_701_pct, feederSpeed_702_pct, feederSpeed_703_pct, feederSpeed_704_pct, f80_cv01_CH1, f80_cv701_CH2, potenciaMotor_CH1, kTon_CH1, kTon_CH2, OSS_CH1, OSS_CH2, estado_CH1, estado_CH2, estadoPicaRoca_CH1, estadoPicaRoca_CH2, supresorPolvo_CH1, ...]
Index: []

[0 rows x 411 columns]

In [47]:
# agujero confirmado, entonces retrasamos el reloj:
aux1 = aux[:start]
aux2 = aux[start:]
aux2 = aux2.shift(-60, freq='T')
aux = pd.concat([aux1, aux2])

In [48]:
# buscamos la quinta fecha conflicitva
aux[aux.index.duplicated(keep = False)]
# para el 2018 la fecha es 2018-05-12 

ID estado_motor_cv701  velocidad_cv701_rpm  \
time                                                                   
2018-05-12 23:00:00  1242660        FUNCIONANDO             58.70754   
2018-05-12 23:01:00  1242661        FUNCIONANDO             59.88436   
2018-05-12 23:02:00  1242662        FUNCIONANDO             59.72718   
2018-05-12 23:03:00  1242663        FUNCIONANDO             59.42297   
2018-05-12 23:04:00  1242664        FUNCIONANDO             58.84817   
...                      ...                ...                  ...   
2018-05-12 23:55:00  1242775        FUNCIONANDO             59.08547   
2018-05-12 23:56:00  1242776        FUNCIONANDO             58.66097   
2018-05-12 23:57:00  1242777        FUNCIONANDO             59.00270   
2018-05-12 23:58:00  1242778        FUNCIONANDO             59.78499   
2018-05-12 23:59:00  1242779        FUNCIONANDO             58.48879   

                     velocidad_cv701  pesometro_cv701  pesometro_cv01  \
time                                                                    
2018-05-12 23:00:00         989.4891         4811.385       2401.4020   
2018-05-12 23:01:00         991.6729         5059.594         -9.3164   
2018-05-12 23:02:00         988.5282         6129.011        283.1284   
2018-05-12 23:03:00         990.5059         4366.351      10668.2700   
2018-05-12 23:04:00         989.0116         5174.062       2597.2840   
...                              ...              ...             ...   
2018-05-12 23:55:00         988.9272         5027.844       5676.8930   
2018-05-12 23:56:00         984.8921         8777.815       6453.6860   
2018-05-12 23:57:00         985.4105         7917.861       6667.1770   
2018-05-12 23:58:00         988.1536         5648.410       8703.9700   
2018-05-12 23:59:00         987.8192         5756.723       8215.5010   

                     velocidadCero_cv01  velocidadCero_A_cv01  \
time                                                            
2018-05-12 23:00:00            99.98879              96.61727   
2018-05-12 23:01:00           100.37380              96.86200   
2018-05-12 23:02:00            97.89288              95.23546   
2018-05-12 23:03:00            97.45527              94.80212   
2018-05-12 23:04:00            98.61527              95.82731   
...                                 ...                   ...   
2018-05-12 23:55:00            98.56305              95.53423   
2018-05-12 23:56:00            98.66575              95.63828   
2018-05-12 23:57:00            97.95366              95.38786   
2018-05-12 23:58:00            97.91531              94.77895   
2018-05-12 23:59:00            98.16232              95.00091   

                     presionHidroset_cr01  potenciaActiva_A_motor_CH1  ...  \
time                                                                   ...   
2018-05-12 23:00:00              209.3885                   101.04830  ...   
2018-05-12 23:01:00              206.9098                    95.73357  ...   
2018-05-12 23:02:00              255.0723                   156.05800  ...   
2018-05-12 23:03:00              239.8495                   110.28890  ...   
2018-05-12 23:04:00              326.8416                   439.13760  ...   
...                                   ...                         ...  ...   
2018-05-12 23:55:00              295.7635                   285.22730  ...   
2018-05-12 23:56:00              286.9901                   365.34080  ...   
2018-05-12 23:57:00              269.0976                   221.28580  ...   
2018-05-12 23:58:00              269.1360                   175.04080  ...   
2018-05-12 23:59:00              276.1839                   229.70650  ...   

                     tempDevanadoAfd21_1F3_cv07  tempDevanadoAfd21_2F3_cv07  \
time                                                                          
2018-05-12 23:00:00                    67.16682                    70.19524   
2018-05-12 23:01:00                    67

In [49]:
# quinta fecha conflictiva: 2018-05-12
end = '2018-05-12 22:59:59'
target = '2018-05-12 23:00:00'
target2 = '2018-05-12 23:59:59'
dfrom = '2018-05-13 00:00:00'
aux1 = aux[:end].copy()
aux2 = aux[target:target2].copy()
aux2_1 = aux2.head(60).copy()
aux2_2 = aux2.tail(60).copy()
aux3 = aux[dfrom:].copy()

aux_f = pd.concat([aux2_2, aux3])

aux_f = aux_f.shift(60, freq='T')

aux = pd.concat([aux1, aux2_1, aux_f])

In [50]:
# sexta fecha conflictiva 2018-08-11, todo esta corrido una hora hacia adelante
# confirmando el agujero:
start = '2018-08-12 01:00:00'
end   = '2018-08-12 01:59:59'
aux[start:end]

Empty DataFrame
Columns: [ID, estado_motor_cv701, velocidad_cv701_rpm, velocidad_cv701, pesometro_cv701, pesometro_cv01, velocidadCero_cv01, velocidadCero_A_cv01, presionHidroset_cr01, potenciaActiva_A_motor_CH1, posicionCono_CH1, corrientePromedio_CH1, T_ambiente_enfriadores_CH1, presionHidroset_cr701, potencia_CH701, posicionCono_CH2, cap_termica_motorCH2, cap_termica_motorCH1, corrientePromedio_CH2, T_ambiente_enfriadores_CH2, potenciaActivaMotor_cv01A, potenciaActivaMotor_cv01B, corrienteMotor_cv01A, corrienteMotor_cv01B, potencia_M1_cv701, potencia_M2_cv701, corrienteMotorA_cv701_faseA, corrienteMotorB_cv701_faseA, corrienteMotorA_cv701_faseB, corrienteMotorB_cv701_faseB, corrienteMotorA_cv701_fasec, corrienteMotorB_cv701_faseC, alimentacionFeeder_cv05, tonelajePromedio_cv05, inventarioStockPilePlanta, alturaPilaDescarga, velocidadFeeder_05, velocidadFeeder_06, velocidadFeeder_07, velocidadFeeder_08, feeder05_onOff, feeder06_onOff, feeder07_onOff, feeder07_onOff.1, TpH_SAG1, TpH_SAG2, TpH_SAG3, stockPile_nivelSensor1, stockPile_nivelSensor2, stockPile_nivel, stockPile_nivelDesc1, stockPile_nivelDesc2, stockPile_nivelDesc3, stockPile_nivelDesc4, stockPile_nivelDesc5, stockPile_nivelDesc6, stockPile_nivelDesc7, stockPile_nivelDesc8, stockPile_nivel7, stockPile_nivel8, stockPile_nivel9, stockPile_nivel10, stockPileMina_CH1, stockPileMina_CH2, feederSpeed_15_onOff, feederSpeed_16_onOff, feederSpeed_17_onOff, feederSpeed_18_onOff, feederSpeed_15_pct, feederSpeed_16_pct, feederSpeed_17_pct, feederSpeed_18_pct, feederSpeed_19_onOff, feederSpeed_20_onOff, feederSpeed_21_onOff, feederSpeed_22_onOff, feederSpeed_19_pct, feederSpeed_20_pct, feederSpeed_21_pct, feederSpeed_22_pct, feederSpeed_701_onOff, feederSpeed_702_onOff, feederSpeed_703_onOff, feederSpeed_704_onOff, feederSpeed_701_pct, feederSpeed_702_pct, feederSpeed_703_pct, feederSpeed_704_pct, f80_cv01_CH1, f80_cv701_CH2, potenciaMotor_CH1, kTon_CH1, kTon_CH2, OSS_CH1, OSS_CH2, estado_CH1, estado_CH2, estadoPicaRoca_CH1, estadoPicaRoca_CH2, supresorPolvo_CH1, ...]
Index: []

[0 rows x 411 columns]

In [51]:
# agujero confirmado, entonces retrasamos el reloj:
aux1 = aux[:start]
aux2 = aux[start:]
aux2 = aux2.shift(-60, freq='T')
aux = pd.concat([aux1, aux2])

In [54]:
aux[aux.index.duplicated(keep = False)]

Empty DataFrame
Columns: [ID, estado_motor_cv701, velocidad_cv701_rpm, velocidad_cv701, pesometro_cv701, pesometro_cv01, velocidadCero_cv01, velocidadCero_A_cv01, presionHidroset_cr01, potenciaActiva_A_motor_CH1, posicionCono_CH1, corrientePromedio_CH1, T_ambiente_enfriadores_CH1, presionHidroset_cr701, potencia_CH701, posicionCono_CH2, cap_termica_motorCH2, cap_termica_motorCH1, corrientePromedio_CH2, T_ambiente_enfriadores_CH2, potenciaActivaMotor_cv01A, potenciaActivaMotor_cv01B, corrienteMotor_cv01A, corrienteMotor_cv01B, potencia_M1_cv701, potencia_M2_cv701, corrienteMotorA_cv701_faseA, corrienteMotorB_cv701_faseA, corrienteMotorA_cv701_faseB, corrienteMotorB_cv701_faseB, corrienteMotorA_cv701_fasec, corrienteMotorB_cv701_faseC, alimentacionFeeder_cv05, tonelajePromedio_cv05, inventarioStockPilePlanta, alturaPilaDescarga, velocidadFeeder_05, velocidadFeeder_06, velocidadFeeder_07, velocidadFeeder_08, feeder05_onOff, feeder06_onOff, feeder07_onOff, feeder07_onOff.1, TpH_SAG1, TpH_SAG2, TpH_SAG3, stockPile_nivelSensor1, stockPile_nivelSensor2, stockPile_nivel, stockPile_nivelDesc1, stockPile_nivelDesc2, stockPile_nivelDesc3, stockPile_nivelDesc4, stockPile_nivelDesc5, stockPile_nivelDesc6, stockPile_nivelDesc7, stockPile_nivelDesc8, stockPile_nivel7, stockPile_nivel8, stockPile_nivel9, stockPile_nivel10, stockPileMina_CH1, stockPileMina_CH2, feederSpeed_15_onOff, feederSpeed_16_onOff, feederSpeed_17_onOff, feederSpeed_18_onOff, feederSpeed_15_pct, feederSpeed_16_pct, feederSpeed_17_pct, feederSpeed_18_pct, feederSpeed_19_onOff, feederSpeed_20_onOff, feederSpeed_21_onOff, feederSpeed_22_onOff, feederSpeed_19_pct, feederSpeed_20_pct, feederSpeed_21_pct, feederSpeed_22_pct, feederSpeed_701_onOff, feederSpeed_702_onOff, feederSpeed_703_onOff, feederSpeed_704_onOff, feederSpeed_701_pct, feederSpeed_702_pct, feederSpeed_703_pct, feederSpeed_704_pct, f80_cv01_CH1, f80_cv701_CH2, potenciaMotor_CH1, kTon_CH1, kTon_CH2, OSS_CH1, OSS_CH2, estado_CH1, estado_CH2, estadoPicaRoca_CH1, estadoPicaRoca_CH2, supresorPolvo_CH1, ...]
Index: []

[0 rows x 411 columns]

In [55]:
aux.to_csv('bd_op_summer.csv')